In [2]:
import numpy as np
import pandas as pd
import patsy
import sklearn.linear_model as linear_model

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV


#### Read in data and conduct an exploratory data analysis.
#### Resolve any data issues you identify and articulate why you did what you did.

In [55]:
sf_crime = pd.read_csv('datasets/sf_crime_train.csv')

In [56]:
sf_crime.head()

## since we will be predicting violent crime versus non-violent crime versus non-crimes
## which are based off of category, we can drop 'descript'
## can also drop address because there could be plenty of formatting variations
## and we have x,y as lat, long

sf_crime.drop(['Descript','Address'], inplace=True, axis=1)

In [57]:
## convert dayofweek to numbers so that if we need to look at data by day,
## we can have the days in the correct order

sf_crime.day = [1 if day == 'Sunday' else 2 if day == 'Monday' else 3 if day == 'Tuesday' else 4 if day == 'Wednesday' else 5 if day == 'Thursday' else 6 if day == 'Friday' else 7 for day in sf_crime.DayOfWeek]
sf_crime.drop('DayOfWeek', inplace=True, axis=1)


#### Create column for hour, month, and year from 'Dates' column.
##### Hint: pd.to_datetime may be helpful.

In [58]:
sf_crime['Dates'] = pd.to_datetime(sf_crime.Dates)

sf_crime['hour'] = sf_crime.Dates.dt.hour
sf_crime['month'] = sf_crime.Dates.dt.month
sf_crime['year'] = sf_crime.Dates.dt.year

### Build a logit model predicting violent crime versus non-violent crime versus non-crimes.

##### We've given you the various crimes that fall into specific categories so you can use these to create these subcategories 

#### Non-Violent Crimes: bad checks, bribery, drug/narcotic, drunkenness, embezzlement, forgery/counterfeiting, fraud, gambling, liquor, loitering, trespass.

#### Non-Crimes: non-criminal, runaway, secondary codes, suspicious occ, warrants.

#### Violent Crimes: everything else.

###### Hint: What type of model do you need here? What should your "baseline" category be?

Need a logistic regression model
baseline = violent crime

#### Fit a model with five folds and lasso regularization
#### Use Cs=15 to test a grid of 15 distinct parameters
#### Remember: Cs describes the inverse of regularization strength

In [75]:
import patsy
nvc = ['BAD CHECKS', 'BRIBERY', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'LIQUOR', 'LOITERING', 'TRESSPASS']
nc = ['NON-CRIMINAL', 'RUNAWAY', 'SECONDARY CODES', 'SUSPICIOUS OCC', 'WARRANTS']

sf_crime['Subcategory'] = ['Non-Violent Crime' if x in nvc else 'Non-Crime' if x in nc else 'Violent Crime' for x in sf_crime['Category']]

sf_crime['NV'] = [1 if x=='Non-Violent Crime' else 0 for x in sf_crime.Subcategory ]
sf_crime['VC'] = [1 if x=='Violent Crime' else 0 for x in sf_crime.Subcategory ]
sf_crime['NC'] = [1 if x=='Non-Crime' else 0 for x in sf_crime.Subcategory ]


In [77]:
X = patsy.dmatrix('~ C(PdDistrict) + C(Resolution)', sf_crime)
X = pd.DataFrame(X, columns=X.design_info.column_names)

y = sf_crime.Subcategory

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)


logreg_cv = LogisticRegressionCV(solver='liblinear', Cs=15, cv=5, penalty='l1',scoring='accuracy')
logreg_cv.fit(X_train, y_train)
logreg_cv.score(X_test, y_test)
y_pred = logreg_cv.predict(X_test)
y_score = logreg_cv.predict_proba(X_test)[:,1]

In [81]:
# find best C per class
print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in enumerate(zip(logreg_cv.C_, logreg_cv.classes_))}
print(best_C)

 best C for class:
{'Violent Crime': 3.7275937203149381, 'Non-Violent Crime': 0.0001, 'Non-Crime': 13.89495494373136}


In [11]:
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns=(logreg_cv.classes_ + ' PREDICTED'), index=(logreg_cv.classes_ + ' TRUE'))
cm

,Non-Crime PREDICTED,Non-Violent Crime PREDICTED,Violent Crime PREDICTED
Non-Crime TRUE,87,0,1181
Non-Violent Crime TRUE,7,0,328
Violent Crime TRUE,43,0,4294


#### Fit regular logit model to 'DRUG/NARCOTIC' and 'BURGLARY' classes; be sure to specify the Y = 1 and Y = 0 classes. Use lasso penalty

In [83]:
import statsmodels.formula.api as sm
import pandas as pd

sf = sf_crime[sf_crime.Category.isin(['DRUG/NARCOTIC','BURGLARY'])]
sf['Subcat_num'] = [1 if x=='Violent Crime' else 0 for x in sf.Subcategory]

X = patsy.dmatrix('~ C(PdDistrict) + C(Resolution)', sf)
X = pd.DataFrame(X, columns=X.design_info.column_names)

y = sf.Subcategory

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)


logreg_cv = LogisticRegressionCV(solver='liblinear', Cs=15, cv=5, penalty='l1')
logreg_cv.fit(X_train, y_train)
logreg_cv.score(X_test, y_test)
y_pred = logreg_cv.predict(X_test)

# find best C per class
print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in enumerate(zip(logreg_cv.C_, logreg_cv.classes_))}
print(best_C)
print 'Classification report:\n', classification_report(y_test, y_pred)

best C for class:
{'Non-Violent Crime': 1.0}
Classification report:
                   precision    recall  f1-score   support

Non-Violent Crime       0.77      0.90      0.83       157
    Violent Crime       0.93      0.83      0.88       249

      avg / total       0.87      0.86      0.86       406



/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Build confusion matrices for the models above


#### Run gridsearch using GridSearchCV and 5 folds
#### Score on accuracy; what does this metric tell us?

In [84]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],'penalty':['l1','l2'] }
clf = GridSearchCV(LogisticRegression(), param_grid, cv=5,scoring='accuracy')
clf = clf.fit(X_train, y_train)

In [85]:
clf.best_score_

0.86374695863746964

#### Find the best parameter

In [86]:
clf.best_params_

{'C': 1, 'penalty': 'l1'}

#### Use this parameter to .fit, .predict, and print a classification_report (we've already imported this for you) for our X and Y

In [87]:
logreg_gs = LogisticRegression(penalty='l1',C=1).fit(X_train, y_train)
logreg_gs.score(X_test, y_test)
y_pred = logreg_gs.predict(X_test)

print 'Accuracy Score: ',accuracy_score(y_test, y_pred)
print 
print 'Classification report:\n', classification_report(y_test, y_pred)

Accuracy Score:  0.85960591133

Classification report:
                   precision    recall  f1-score   support

Non-Violent Crime       0.77      0.90      0.83       157
    Violent Crime       0.93      0.83      0.88       249

      avg / total       0.87      0.86      0.86       406



#### When looking at the classification report, remember:
##### Precision = True Positives / (True Positives + False Positives)

A precision score of 1 indicates that the classifier never mistakenly added observations from another class. A precision score of 0 would mean that the classifier misclassified every instance of the current class.
 
##### Recall = True Positives / (True Positives + False Negatives)
A recall score of 1 indicates that the classifier correctly predicted (found) all observations of the current class (by implication, no false negatives, or misclassifications of the current class). A recall score of 0 alternatively means that the classifier missed all observations of the current class.

##### F1-Score = 2 * (Precision * Recall) / (Precision + Recall)
The f1-score's best value is 1 and worst value is 0, like the precision and recall scores. It is a useful metric for taking into account both measures at once. 

##### Support is simply the number of observations of the labelled class.

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html